In [1]:
import pandas as pd

In [3]:
! wget https://synthetichealth.github.io/synthea-sample-data/downloads/synthea_sample_data_csv_apr2020.zip

--2023-09-06 16:44:51--  https://synthetichealth.github.io/synthea-sample-data/downloads/synthea_sample_data_csv_apr2020.zip
Resolving synthetichealth.github.io (synthetichealth.github.io)... 185.199.111.153, 185.199.108.153, 185.199.109.153, ...
Connecting to synthetichealth.github.io (synthetichealth.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8982431 (8.6M) [application/zip]
Saving to: ‘synthea_sample_data_csv_apr2020.zip’

synthea_sample_data 100%[===================>]   8.57M  3.16MB/s    in 2.7s    

2023-09-06 16:44:54 (3.16 MB/s) - ‘synthea_sample_data_csv_apr2020.zip’ saved [8982431/8982431]



In [4]:
! unzip synthea_sample_data_csv_apr2020.zip

Archive:  synthea_sample_data_csv_apr2020.zip
   creating: csv/
  inflating: csv/medications.csv     
  inflating: csv/providers.csv       
  inflating: csv/payer_transitions.csv  
  inflating: csv/imaging_studies.csv  
  inflating: csv/supplies.csv        
  inflating: csv/payers.csv          
  inflating: csv/allergies.csv       
  inflating: csv/procedures.csv      
  inflating: csv/organizations.csv   
  inflating: csv/conditions.csv      
  inflating: csv/careplans.csv       
  inflating: csv/encounters.csv      
  inflating: csv/devices.csv         
  inflating: csv/immunizations.csv   
  inflating: csv/patients.csv        
  inflating: csv/observations.csv    


In [5]:
df = pd.read_csv("csv/observations.csv")
df.head()

,DATE,PATIENT,ENCOUNTER,CODE,DESCRIPTION,VALUE,UNITS,TYPE
0,2012-01-23T17:45:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,8302-2,Body Height,193.3,cm,numeric
1,2012-01-23T17:45:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,72514-3,Pain severity - 0-10 verbal numeric rating [Sc...,2.0,{score},numeric
2,2012-01-23T17:45:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,29463-7,Body Weight,87.8,kg,numeric
3,2012-01-23T17:45:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,39156-5,Body Mass Index,23.5,kg/m2,numeric
4,2012-01-23T17:45:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,8462-4,Diastolic Blood Pressure,82.0,mm[Hg],numeric


In [11]:
df.PATIENT.value_counts().describe().round(0).astype(int)

count    1171
mean      256
std       501
min        34
25%       134
50%       164
75%       244
max      9023
Name: PATIENT, dtype: int64

In [57]:
df[df.PATIENT==df.PATIENT.unique()[1]][["DESCRIPTION", "VALUE", "UNITS"]][:30].values[24]

array(['Tobacco smoking status NHIS', 'Never smoker', nan], dtype=object)

In [76]:
observations = ", ".join([" ".join(el) for el in df[df.PATIENT==df.PATIENT.unique()[1]][["DESCRIPTION", "VALUE", "UNITS"]][:30].values[:24]])

In [81]:
CLARIFAI_PAT="" # Place your Clarifai Personal Access Token here

In [82]:
import os
import streamlit as st
from langchain.llms import Clarifai
from langchain import PromptTemplate, LLMChain

# https://python.langchain.com/docs/integrations/llms/clarifai
clarifai_llm = Clarifai(
    pat=CLARIFAI_PAT, user_id='openai', app_id='chat-completion', model_id='GPT-3_5-turbo'
)

template = """A patient has the following list of measurements: {observations}

The following measurements are out of normal range: """

prompt = PromptTemplate(template=template, input_variables=["observations"])

In [83]:
# Create LLM chain
llm_chain = LLMChain(prompt=prompt, llm=clarifai_llm)

In [84]:
result = llm_chain.run(observations)

In [85]:
print(result)

- Body Mass Index: 29.3 kg/m2 (overweight)
- Diastolic Blood Pressure: 87.0 mm[Hg] (elevated)
- Total Cholesterol: 174.1 mg/dL (borderline high)
- Low Density Lipoprotein Cholesterol: 86.9 mg/dL (borderline high)
- Erythrocyte distribution width: 45.7 fL (high)
- Platelet distribution width: 440.7 fL (high)
